In [4]:
import itertools
import re
import math

import numpy as np
import networkx as nx

from functools import lru_cache
from collections import Counter, defaultdict
from aocd.transforms import lines, numbers
from lib import imag2tup, tup2imag, level_a, level_b, level_ab, setdata, testdata

setdata(["199\n200\n208\n210\n200\n207\n240\n269\n260\n263", 'forward 5\ndown 5\nforward 8\nup 3\ndown 8\nforward 2',
        """asdf"""])

In [6]:
@level_ab(1)
def solve(data, method=0):
    data = np.array(numbers(data))
    if method:
        data = data[2:] + data[1:-1] + data[:-2]  # 3-frame-windowing
    return ( ( data[1:] - data[:-1] ) > 0).sum()

Ex. 1a	7
Lvl 1a	1184
Ex. 1b	5
Lvl 1b	1158


In [11]:
@level_ab(2)
def solve(data, method=1):
    DIRS = {"forward": 1j, "up": 1, "down": -1}
    pos = aim = 0
    for line in data.split("\n"):
        dir, dist = line.split()
        dir, dist = DIRS[dir], int(dist)
        if method:
            if dir.real:
                aim += dist * dir
            else:
                pos += dist + dist * aim * dir
        else:
            pos += dir * dist
    return np.absolute(imag2tup(pos)).prod()

Ex. 2a	150
Lvl 2a	1694130
Ex. 2b	900
Lvl 2b	1698850445


In [8]:
@level_ab(3)
def solve(data, method=1):
    pos = aim = 0
    for line in lines(data):
        pos += int(line)
    return pos

Ex. 2a	150
Lvl 2a	1694130
Ex. 2b	900
Lvl 2b	1698850445


In [ ]:
@level_ab(4)
def solve(data, method=1):
    pos = aim = 0
    for line in lines(data):
        pos += int(line)
    return pos